In [1]:
from PIL import Image
import urllib.request
import io
from skimage import io
from ipywidgets import Image
import pandas as pd
import time 
import ast
from stack_functions import *
import requests

In [6]:
def get_address(query):
        base = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input='
        locationbias= '42.3314584645106, -71.1039191294055'
        key = "YOUR API KEY"
        query = query
        fields = 'formatted_address,name,geometry'
        inputtype = 'textquery'
        
        url_search = base + query + '&inputtype=' + inputtype + '&fields=' + fields + '&locationbias=circle:2000@' + locationbias + '&key=' + key
        my_dict = requests.get(url_search).json()
        return my_dict

In [7]:
def get_commute(dest, origin):
    
    base = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&'
    origin = 'origins=' + origin + '&'
    dest_str = 'destinations=' + dest + '&'
    api_key = 'key=YOUR API KEY'
    
    url_search = base + origin + dest_str + api_key
    my_dict = requests.get(url_search).json()
    return my_dict

In [42]:
def stack_scraper(results, city, job_title):
    num = -1
    file = 1
    SKIPPER = 0 # Skips this many of the first jobs
    col_list = ['title', 'company', 'location', 'salary', 'link', 'pulldate','tags', 'id']
    job_df = pd.DataFrame(columns=col_list, index=range(0,results))
    
    if(file > SKIPPER):
        # for results
        for start in range(0, results, 10):
            temp_url = 'https://stackoverflow.com/jobs?q=' + job_title +'&sort=p'
            page = requests.get(temp_url) 
            time.sleep(1)  
            soup = get_soup(page.text)
            divs = soup.find_all(name="div", attrs={"class":"-job-summary"})
            # if results exist
            if(len(divs) == 0):
                break
            
            for div in divs:
                #if (extract_date(div) is True):
                    #specifying row num for index of job posting in dataframe
                    num = num + 1
                    job_df.loc[num, 'pulldate'] = str(time.strftime("%m-%d-%Y_%I-%M_%p", time.localtime(time.time())))
                    job_df.loc[num, 'link'] = extract_title_link(div)[1]
                    job_df.loc[num, 'title'] = extract_title_link(div)[0]
                    job_df.loc[num, 'company'] = extract_comp_location(div)[0]
                    #job_df.loc[num, 'summary'] = extract_summary(div)
                    job_df.loc[num, 'salary'] = extract_salary(div)
                    job_df.loc[num, 'location'] = extract_comp_location(div)[1]
                    job_df.loc[num, 'tags'] = extract_tags(div)
                    job_df.loc[num, 'id'] = extract_id(div)
                    job_df.loc[num, 'comp_loc'] = extract_comp_location(div)[0] + ', ' + extract_comp_location(div)[1]
    #job_df = job_df.dropna(how='all')
    companies = job_df['comp_loc']
    ids = job_df['id']
    comm_df = pd.DataFrame(columns=['company', 'place', 'address', 'distance', 'duration'])
    
    for index, company in companies.items():
        tmp_id = ids[index]
        text_dict = get_address(company)
        if text_dict['status'] == 'ZERO_RESULTS':
            print(company + ' has 0 results in get_address.')
            continue
        else:
            address = text_dict['candidates'][0]['formatted_address']
        place = text_dict['candidates'][0]['name']    
        res_lat_lng = str(text_dict['candidates'][0]['geometry']['location']['lat']) + ',' + str(text_dict['candidates'][0]['geometry']['location']['lng'])

        comm = get_commute(dest=res_lat_lng, origin='8 Oswald Street')
        if comm['rows'][0]['elements'][0]['status'] == 'ZERO_RESULTS':
            print(company + ' has 0 results in get_commute.')
            continue
        else:
            try:
                distance = (comm['rows'][0]['elements'][0]['distance']['text'])
                duration = (comm['rows'][0]['elements'][0]['duration']['text'])
            except:
                print(comm)
                break

        
        comm_df.loc[index, 'company'] = company
        comm_df.loc[index, 'place'] = place
        comm_df.loc[index, 'address'] = address
        comm_df.loc[index, 'distance'] = distance
        comm_df.loc[index, 'duration'] = duration
        comm_df.loc[index, 'id'] = tmp_id
        
    #comm_df['id'] = ids
    fin_df = job_df.merge(comm_df, how = 'left', left_on='id', right_on='id')#.drop('company_y',1)
    file = file + 1
    return fin_df

In [44]:
fetch = stack_scraper(5, job_title= 'python+developer', city='boston')
fetch

stensul,  Palermo, Argentina         has 0 results in get_commute.
Applied Blockchain,  London, UK         has 0 results in get_commute.
Qwil Messenger,  London, UK         has 0 results in get_address.
Sepasoft, Inc.,  El Dorado Hills, CA         has 0 results in get_address.
Kalixa Payments Group,  Wien, Austria         has 0 results in get_commute.
Kalixa Payments Group,  Wien, Austria         has 0 results in get_commute.
eBay,  London Borough of Richmond upon Thames, UK         has 0 results in get_commute.


,title,company_x,location,salary,link,pulldate,tags,id,comp_loc,company_y,place,address,distance,duration
0,Adobe Experience Manager Engineer - Vice Presi...,JP Morgan Chase,"Jersey City, NJ",,https://www.stackoverflow.com/jobs/206300/adob...,10-24-2018_02-49_PM,"java, python, sql,",206300,"JP Morgan Chase, Jersey City, NJ","JP Morgan Chase, Jersey City, NJ",J P Morgan Chase Backup CCC,"575 Washington Blvd, Jersey City, NJ 07310, USA",216 mi,3 hours 54 mins
1,.Net Developer,Greenshades,"Jacksonville, FL",,https://www.stackoverflow.com/jobs/203300/net-...,10-24-2018_02-49_PM,"c#, sql, .net, agile, visual-studio,",203300,"Greenshades, Jacksonville, FL","Greenshades, Jacksonville, FL",Greenshades Software,"7020 A C Skinner Pkwy #100, Jacksonville, FL 3...","1,162 mi",17 hours 17 mins
2,AEM Engineering Lead,JP Morgan Chase,"Chicago, IL",,https://www.stackoverflow.com/jobs/206301/aem-...,10-24-2018_02-49_PM,"java, python, sql,",206301,"JP Morgan Chase, Chicago, IL","JP Morgan Chase, Chicago, IL",JPMORGAN CHASE,"440 S LaSalle St, Chicago, IL 60605, USA",981 mi,14 hours 50 mins
3,DevOps Engineer,"Logi Analytics, Inc.","McLean, VA",$110k - 130k ...,https://www.stackoverflow.com/jobs/200123/devo...,10-24-2018_02-49_PM,"atlassian, virtualization, jenkins, scripting,...",200123,"Logi Analytics, Inc., McLean, VA","Logi Analytics, Inc., McLean, VA",Logi Analytics,"7900 Westpark Dr, A200, McLean, VA 22102, Unit...",449 mi,7 hours 9 mins
4,"Senior Software Engineer, Team Lead","Logi Analytics, Inc.","McLean, VA",$135k - 150k ...,https://www.stackoverflow.com/jobs/200124/seni...,10-24-2018_02-49_PM,"java, database, mongodb, maven, hadoop,",200124,"Logi Analytics, Inc., McLean, VA","Logi Analytics, Inc., McLean, VA",Logi Analytics,"7900 Westpark Dr, A200, McLean, VA 22102, Unit...",449 mi,7 hours 9 mins
5,iOS / Android Developer,Boyle Software,"New York, NY",$100k - 140k ...,https://www.stackoverflow.com/jobs/200609/ios-...,10-24-2018_02-49_PM,"objective-c, cordova, swift, android, ios,",200609,"Boyle Software, New York, NY","Boyle Software, New York, NY","Boyle Software, Inc.","42 W 24th St, New York, NY 10010, USA",213 mi,3 hours 45 mins
6,Senior Software Developer,ManageAmerica Integrated Billing Services,No office location,Remote,https://www.stackoverflow.com/jobs/197461/seni...,10-24-2018_02-49_PM,"c#, .net, sql, angular,",197461,"ManageAmerica Integrated Billing Services, No...","ManageAmerica Integrated Billing Services, No...",Medical Healthcare Solutions,"300 Brickstone Square, Andover, MA 01810, USA",29.5 mi,40 mins
7,Senior QA Automation,stensul,"Palermo, Argentina",,https://www.stackoverflow.com/jobs/201779/seni...,10-24-2018_02-49_PM,"qa, selenium, automation,",201779,"stensul, Palermo, Argentina",NaN,NaN,NaN,NaN,NaN
8,Senior SQL Software Engineer,"Jack Henry & Associates, Inc.®","Allen, TX",,https://www.stackoverflow.com/jobs/203299/seni...,10-24-2018_02-49_PM,"sql, jdbc, java, sql-server,",203299,"Jack Henry & Associates, Inc.®, Allen, TX ...","Jack Henry & Associates, Inc.®, Allen, TX ...",Jack Henry & Associates Inc,"2 Gateway Blvd # 140, East Granby, CT 06026, USA",110 mi,1 hour 50 mins
9,Front End Developer,Boyle Software,"New York, NY",$90k - 120k,https://www.stackoverflow.com/jobs/204292/fron...,10-24-2018_02-49_PM,"javascript, reactjs, angular, node.js, vue.js,",204292,"Boyle Software, New York, NY","Boyle Software, New York, NY","Boyle Software, Inc.","42 W 24th St, New York, NY 10010, USA",213 mi,3 hours 45 mins
